## Extraindo os dados das avaliações

### Indentificando as features e values das avaliações

In [ ]:
def extract_data_grade(path):
    
    features, values = [], []
    with open(path, 'r') as fl:
        for line in fl:
            data_grade = re.search(r'(\-{4}\s)(.+)(:\s)(.+)', line)
            if data_grade:
                feature = data_grade.group(2)
                value = data_grade.group(4)
                if 'grade' in feature:
                    feature = feature.split(' ')[0]
                feature = feature.replace(' of ', '_')
                features.append(feature)
                values.append(value)
    return features, values

### Organizando os dados para o armazenamento no banco de dados

In [ ]:
import os
import re
import pymysql

from pprint import pprint
from datetime import datetime

In [ ]:
def connection_database():
    connection = pymysql.connect(host='localhost',
                             user='root',
                             password='open',
                             db='dataset-workbench',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
    return connection

In [ ]:
conn = connection_database()
cursor = conn.cursor()
for semester_id in os.listdir('../dataset/codebench/'):
    path0 = os.path.join('../dataset/codebench/', semester_id)
    for class_id in os.listdir(path0):
            path1 = os.path.join(path0, class_id)
            path2 = os.path.join(path1, 'users')
            for user_id in os.listdir(path2):
                    path3 = os.path.join(path2, user_id)
                    path4 = os.path.join(path3, 'grades')
                    for grade_id in os.listdir(path4):
                        if 'final' not in grade_id:
                            path5 = os.path.join(path4, grade_id)
                            features, values = extract_data_grade(path5)
                            grade_id = grade_id.replace('.log', '')
                            #print(f'semester_id={semester_id}, class_id={class_id}, user_id={user_id}, grade_id={grade_id}, features={features}, values={values}')
                            fk_user = f"'{semester_id}#{class_id}#{user_id}', "
                            fk_assessment = f"'{semester_id}#{class_id}#{grade_id}', "
                            features_ = 'user_id, assessment_id, '
                            values_ = fk_user + fk_assessment
                            for feature, value in zip(features, values):
                                features_ += f'{feature}, '
                                values_ += f'{value}, '
                            insert_user_assessment_sql = f'INSERT INTO user_assessment ({features_[:-2]}) VALUES ({values_[:-2]})'
                            cursor.execute(insert_user_assessment_sql)
conn.commit()
conn.close()